In [27]:
from strategy.financialpredict import FinancialPredict
from processor.processor import Processor as p
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
import pickle
import numpy as np

In [46]:
start_date = datetime(2021,1,1)
end_date = datetime(2022,1,13)
all_recs = []
for category_num in range(3,5,1):
    fp = FinancialPredict(start_date,end_date
                          ,modeling_params={"score_requirement":70,"categories":category_num,"model_training_year":4},
                        trading_params={"requirement":10,"value":True})
    fp.subscribe()
    rec_transforms = fp.rec_transform()
    params = fp.modeling_params
    params["year"] = datetime.now().year
    params["quarter"] = (datetime.now().month - 1) // 3 + 1
    if fp.ismodeled():
        fp.db.connect()
        models = fp.db.query("models",params)
        fp.db.disconnect()
    #     return models
    else:
        market = fp.subscriptions["market"]["db"]
        product_db = fp.subscriptions["stock_category"]["db"]
        categories_nums = fp.modeling_params["categories"]
        market.connect()
        product_db.connect()
        financials = market.retrieve("unified_financials")
        sp5 = market.retrieve("sp500")
        categories = product_db.retrieve("sim")
        product_db.disconnect()
        financials["year"] = [x.year for x in financials["date"]]
        financials["quarter"] = [x.quarter for x in financials["date"]]
        financials["year"] = [row[1]["year"] + 1 if row[1]["quarter"] == 4 else row[1]["year"] for row in financials.iterrows()]
        financials["quarter"] = [1 if row[1]["quarter"] == 4 else row[1]["quarter"] + 1 for row in financials.iterrows()]
        financials = financials.groupby(["year","quarter","ticker"]).mean().reset_index()
        financials["date"] = [datetime(row[1]["year"], row[1]["quarter"] * 3 -  2, 1) for row in financials.iterrows()]
        start = fp.start_date.year
        end = fp.end_date.year
        financial_factors = [x for x in financials.columns if x not in ["year","quarter","ticker","date"]]
        yearly_gap = 1
        fp.db.connect()
        initial_data = fp.rec_transform()
        labels = initial_data.merge(categories,on=["year","quarter","ticker"],how="left")
        factors = financials.merge(categories,on=["year","quarter","ticker"],how="left")
        factors["date"] = [datetime(row[1]["year"], row[1]["quarter"] * 3 -  2, 1) for row in factors.iterrows()]
        model_training_year = fp.modeling_params["model_training_year"]
        sims = []
        year = datetime.now().year
        quarter = (datetime.now().month - 1) // 3 + 1
        if quarter == 1:
            new_quarter = 4
            new_year = year - 1
        else:
            new_quarter = quarter - 1
            new_year = year
        quarterly_categories = labels[(labels["year"]==new_year) & (labels["quarter"]==new_quarter)]
        for category in tqdm(list(quarterly_categories[f"{categories_nums}_classification"].unique())):
            try:
                category_tickers = quarterly_categories[quarterly_categories[f"{categories_nums}_classification"]==category]["ticker"].unique()
                training_factors = factors[factors["ticker"].isin(list(category_tickers))]
                training_factors.sort_values("date",inplace=True)
                if quarter == 1:
                    new_quarter = 4
                    new_year = year - 1
                else:
                    new_quarter = quarter - 1
                    new_year = year
                last_index = training_factors[(training_factors["year"] == new_year) & (training_factors["quarter"]==new_quarter)].index.values.tolist()[0]
                spliced_training = training_factors.iloc[:last_index]
                relevant_factors = spliced_training[(spliced_training["year"] < year) & (spliced_training["year"]>=year-model_training_year)]
                training_labels = labels[labels["ticker"].isin(list(category_tickers))]
                training_labels.sort_values("date",inplace=True)
                last_index = training_labels[(training_labels["year"] == new_year) & (training_labels["quarter"]==new_quarter)].index.values.tolist()[0]
                training_labels = training_labels.iloc[:last_index]
                relevant_labels = training_labels[(training_labels["year"] < year) & (training_labels["year"]>=year-model_training_year)]
                relevant = relevant_factors.merge(relevant_labels,on=["year","quarter","ticker"],how="left").dropna()
                relevant.reset_index(inplace=True)
                X = relevant[financial_factors]
                y = relevant["adjclose"]
                models = m.regression({"X":X,"y":y})
                models["year"] = year
                models["quarter"] = quarter
                models["category"] = category
                models["factors"] = [financial_factors for i in range(models.index.size)]
                models["model"] = [pickle.dumps(x) for x in models["model"]]
                for param in fp.modeling_params:
                    models[param] = fp.modeling_params[param]
                fp.db.connect()
                fp.db.store("models",models)
                fp.db.disconnect()
                sims.append(models)
            except Exception as  e:
                print(year,quarter,category,str(e))
    params = fp.modeling_params
    params["year"] = datetime.now().year
    params["quarter"] = (datetime.now().month - 1) // 3 + 1
    if fp.ismodeled():
        fp.db.connect()
        models = fp.db.query("models",params)
        fp.db.disconnect()
    else:
        models = fp.model()
    market = fp.subscriptions["market"]["db"]
    product_db = fp.subscriptions["stock_category"]["db"]
    categories_nums = fp.modeling_params["categories"]
    market.connect()
    product_db.connect()
    financials = market.retrieve("unified_financials")
    sp5 = market.retrieve("sp500")
    categories = product_db.retrieve("sim")
    product_db.disconnect()
    financials["year"] = [x.year for x in financials["date"]]
    financials["quarter"] = [x.quarter for x in financials["date"]]
    financials["year"] = [row[1]["year"] + 1 if row[1]["quarter"] == 4 else row[1]["year"] for row in financials.iterrows()]
    financials["quarter"] = [1 if row[1]["quarter"] == 4 else row[1]["quarter"] + 1 for row in financials.iterrows()]
    financials = financials.groupby(["year","quarter","ticker"]).mean().reset_index()
    financials["date"] = [datetime(row[1]["year"], row[1]["quarter"] * 3 -  2, 1) for row in financials.iterrows()]
    financial_factors = [x for x in financials.columns if x not in ["year","quarter","ticker","date"]]
    yearly_gap = 1
    fp.db.connect()
    factors = financials.merge(categories,on=["year","quarter","ticker"],how="left")
    factors["date"] = [datetime(row[1]["year"], row[1]["quarter"] * 3 -  2, 1) for row in factors.iterrows()]
    model_training_year = fp.modeling_params["model_training_year"]
    sims = []
    year = datetime.now().year
    quarter = (datetime.now().month - 1) // 3 + 1
    if quarter == 1:
        new_quarter = 4
        new_year = year - 1
    else:
        new_quarter = quarter - 1
        new_year = year
    quarterly_categories = categories[(categories["year"]==new_year) & (categories["quarter"]==new_quarter)]
    models["model"] = [pickle.loads(x) for x in models["model"]]
    for category in list(quarterly_categories[f"{categories_nums}_classification"].unique()):
        try:
            category_tickers = quarterly_categories[quarterly_categories[f"{categories_nums}_classification"]==category]["ticker"].unique()
            training_factors = factors[factors["ticker"].isin(list(category_tickers))]
            training_factors.sort_values("date",inplace=True)
            prediction_data = training_factors[(training_factors["year"]==new_year) & (training_factors["quarter"]==new_quarter)]
            sim = prediction_data
            sim = sim.groupby(["year","quarter","ticker"]).mean().reset_index()
            category_models = models[models["category"]==category]
            for i in range(category_models.index.size):
                model = category_models.iloc[i]
                api = model["api"]
                score = model["score"]
                financial_factors = model["factors"]
                if score >= fp.modeling_params["score_requirement"]/100:
                    sim[f"{api}_prediction"] = model["model"].predict(sim[financial_factors])
                    sim[f"{api}_score"] = model["score"].item()
            sim["categories"] = categories_nums
            final_cols = ["ticker","categories"]
            final_cols.extend([x for x in list(sim.columns) if "prediction" in x or "score" in x])
            sim = sim[final_cols]
            sim.fillna(0,inplace=True)
            sim["prediction"] = [np.nanmean([row[1][x] for x in sim.columns if "prediction" in x and row[1][x] != 0]) for row in sim.iterrows()]
            sim["score"] = [np.nanmean([row[1][x] for x in sim.columns if "score" in x and row[1][x] != 0]) for row in sim.iterrows()]
            for param in params:
                sim[param]=params[param]
            if sim.index.size > 1:
                fp.db.store("recs",sim)
                sims.append(sim)
        except Exception as e:
            print(year,quarter,category,str(e)) 
            continue
    fp.db.disconnect()
    market.disconnect()
    recs = pd.concat(sims)
    market.connect()
    whut = []
    for ticker in tqdm(recs["ticker"]):
        ticker_data = market.retrieve_ticker_prices("prices",ticker)
        ticker_data = p.column_date_processing(ticker_data)
        ticker_data["year"] = [x.year for x in ticker_data["date"]]
        ticker_data["quarter"] = [x.quarter for x in ticker_data["date"]]
        whut.append(ticker_data)
    market.disconnect()
    hmm = pd.concat(whut)
    final = hmm.merge(recs,on=["year","quarter","ticker"],how="left").dropna().reset_index(drop=True)
    final["delta"] = (final["prediction"] - final["adjclose"]) / final["adjclose"]
    completed_recs = final.copy()
    completed_recs["category_num"] = category_num
    all_recs.append(completed_recs)

 13%|███████████████▍                                                                                                      | 66/505 [00:07<00:38, 11.40it/s]

BRK.B 'date'


 16%|███████████████████▏                                                                                                  | 82/505 [00:08<00:32, 13.00it/s]

BF.B 'date'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [00:52<00:00,  9.67it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [02:22<00:00,  4.44s/it]


2022 1 nan list index out of range


  0%|                                                                                                                                | 0/11 [00:00<?, ?it/s]

2022 1 nan list index out of range


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 429/429 [00:43<00:00,  9.87it/s]


In [47]:
ar = pd.concat(all_recs)

In [50]:
heheh = ar[(ar["delta"]>0.1) & (ar["date"]>datetime(2022,1,11))][["date","ticker","prediction","adjclose","delta","category_num"]].sort_values("delta",ascending=False)
heheh.sort_values("delta",ascending=False)

,date,ticker,prediction,adjclose,delta,category_num
111,2022-01-12,HBAN,175.078491,17.09,9.244499,4
143,2022-01-12,KEY,114.829533,26.40,3.349604,4
7,2022-01-12,AFL,202.064738,63.34,2.190160,4
215,2022-01-12,RF,65.750053,24.78,1.653352,4
151,2022-01-12,MRK,134.376272,81.20,0.654880,4
399,2022-01-12,BAC,64.616813,48.86,0.322489,4
127,2022-01-12,INTC,69.736453,55.74,0.251102,4
23,2022-01-12,AMP,399.235820,328.20,0.216441,4
7,2022-01-12,CHTR,709.670578,613.98,0.155853,3
431,2022-01-12,MS,115.462087,103.05,0.120447,4
